<a href="https://colab.research.google.com/github/AI-Ahmed/movie-reviewer/blob/main/Movie_Reviewer_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proteniea Challenge

## Dependencies

- Here, I'm going to install the dependencies that are helping me to build my pipeline. I'm using **Hay Stack** – it is an **open-source framework** for building search systems that work intelligently over large document collections.

- I wanted to use **ElasticSearch** for saving the documents after finishing the sentiment analysis but I have not gotten any answer from you!

- For **Language Detection**, the library you mentioned was having an error in reading the length of the sequence while the model is trying to pack padding the sequence of embedding to the LSTMs, I **Pulled** the package, and fixed the issue that occurred.



In [ ]:
#@title (A) Check the GPU Hardware Specifications
!nvidia-smi

Thu Mar 31 10:33:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title ### (B) Installing Libraries
#@markdown <br>

#@markdown ####(1) Haystack framework

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# !pip install -q git+https://github.com/deepset-ai/haystack.git@a1040a17b2697bbaea2585dbcbf7cd175ce8cb75#egg=farm-haystack[colab]

# For FAISS DocumentStore
# !pip install -q git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

#@markdown ---
#@markdown ####(2) ElasticSearch DocumentStore
# In Colab / No Docker environments: Start Elasticsearch from source

# ! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
# ! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
# ! chown -R daemon:daemon elasticsearch-7.9.2

# import os
# from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(
#     ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
# )
# # wait until ES has started
# ! sleep 30
#@markdown ---
#@markdown ####(3) Install pygraphviz
# Install pygraphviz
!apt install -q libgraphviz-dev
!pip install -q pygraphviz

#@markdown ---
#@markdown ####(4) install Language Detection Repos
!pip install git+https://github.com/AI-Ahmed/seqtolang

#@markdown ---
#@markdown ####(5) install Fire
!pip install -q Fire

     |████████████████████████████████| 2.1 MB 13.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-2x2hd1ql/farm-haystack_d8907350da2d451e987bc3838c6dbaf5
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-2x2hd1ql/farm-haystack_d8907350da2d451e987bc3838c6dbaf5
  Resolved https://github.com/deepset-ai/haystack.git to commit 57bb8c41311d31f03d29bc75be13d4cc244c208c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB

In [ ]:
#@title (C) Connecting with Drive and create Main Directory
#@markdown Connect with GDrive if the dataset in GDrive
import os
if not os.path.isdir('movie_reviewer'):
  os.mkdir('movie_reviewer')
connect_with_gdrive = True #@param {type: "boolean"}
if connect_with_gdrive:
  from google.colab import drive
  from IPython.display import clear_output
  print("Connecting with GDrive...")
  try:
    drive.mount('/content/drive')
    clear_output(wait=True)
    print('Successfully Connected with GDrive')
  except Exception as e:
    print(f"Failed to connect with Google Drive, Reason: {e}")
    

Successfully Connected with GDrive


**All The Environment variables Initialized here in `utils.py`. Please, change it according to your AWS Environment.**

In [ ]:
#@title (D) Libraries
# %%writefile movie_reviewer/utils.py
import os
import sys
import json
import warnings
from glob import glob
from collections import OrderedDict
from pprint import pprint
from typing import Text, List, Dict, Tuple, Optional, Union
warnings.filterwarnings("ignore")

import fire

import pymysql
from sqlalchemy import create_engine
import pandas as pd
from tqdm.auto import trange, tqdm

# Language Detector
from seqtolang import Detector

# Document Store
from haystack.utils import (launch_es,
                            print_answers,
                            print_documents,
                            convert_files_to_docs,
                            clean_wiki_text)

# Composer
from haystack import BaseComponent, Pipeline

# Schema
from haystack.schema import Document

# Nodes
## Translator
from haystack.nodes import TransformersTranslator, TransformersDocumentClassifier

## Sentiment Analysis
from haystack.nodes import TransformersDocumentClassifier

# Disable Telemery
from haystack.telemetry import disable_telemetry
disable_telemetry()

# Environment Configurations
os.environ['HAYSTACK_TELEMETRY_ENABLED'] = "False"
os.environ['ENDPOINT'] = "database-1.c0tbaqotqgli.us-west-1.rds.amazonaws.com"
os.environ['USER'] = "admin"
os.environ['PASS'] = "adminadmin"
os.environ['DATABASE'] = "database-1"

INFO - haystack.telemetry -  Telemetry has been disabled.


In [ ]:
#@title (F) Table Data Configuration
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

## Preprocessing

- In the preprocessing, I had to pull the samples you sent to us and start to create synthetic data out of them so that I can simulate the amount of batched data you're going to test the pipeline using it. 

In [ ]:
en_text = "Best movie of all time !"

In [ ]:
dutch_text = "Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht entt\u00e4uscht - \u00fcberhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind gro\u00dfartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer gro\u00dfartigen Uhr zu machen!"

In [ ]:
french_text = "C'est une critique de film neutre, mais je n'aime pas non plus Leon."

### Batch Example

In [ ]:
list_of_d = []
for f in glob('/content/drive/MyDrive/Work/Proteinea Challenge/*.json'):
  data = json.load(open(f, 'r'))
  list_of_d.append(data)

In [ ]:
batch_data = {}
for d in list_of_d:
  for k, v in d.items():
    batch_data.setdefault(k, []).append(v)

In [ ]:
for key, value in batch_data.items():
  batch_data[key] = value * 143

In [ ]:
with open('/content/drive/MyDrive/Work/Proteinea Challenge/experimentation/synthatic_batch_data.json', 'w') as js:
  json.dump(batch_data, js, indent=2)

In [ ]:
df = pd.DataFrame.from_dict(batch_data)
df

,_id,movie_name,release_year,producer,director,review_text,user_name
0,sample_3,The Room,2003,Wiseau-Films,Tommy Wiseau,"If this is a movie, then I am PhD in Artificial Intelligence.",alpha_helix
1,sample_0,Squid Game,2021,Siren Pictures Inc,Hwang Dong-hyuk,"Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!",kosmasp
2,sample_1,The Matrix,1999,"Lana Wachowski,James McTeiguem,Grant Hill","Lana Wachowski,Lilly Wachowski",Best movie of all time !,abs_waleedm
3,sample_4,Lion King,1994,"Don Hahn,Alice Dewey,Sarah McArthur","Rob Minkoff,Roger Allers",They don't make Disney movies like that anymore. I cry every time I watch it.,beta_sheet
4,sample_2,Léon: The Professional,1994,Patrice Ledoux,Luc Besson,"C'est une critique de film neutre, mais je n'aime pas non plus Leon.",neu_star
...,...,...,...,...,...,...,...
710,sample_3,The Room,2003,Wiseau-Films,Tommy Wiseau,"If this is a movie, then I am PhD in Artificial Intelligence.",alpha_helix
711,sample_0,Squid Game,2021,Siren Pictures Inc,Hwang Dong-hyuk,"Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!",kosmasp
712,sample_1,The Matrix,1999,"Lana Wachowski,James McTeiguem,Grant Hill","Lana Wachowski,Lilly Wachowski",Best movie of all time !,abs_waleedm
713,sample_4,Lion King,1994,"Don Hahn,Alice Dewey,Sarah McArthur","Rob Minkoff,Roger Allers",They don't make Disney movies like that anymore. I cry every time I watch it.,beta_sheet


In [ ]:
for row in df.values.tolist():
  x = dict(zip(df.columns.values.tolist(), row))
  print(x)

{'_id': 'sample_3', 'movie_name': 'The Room', 'release_year': 2003, 'producer': 'Wiseau-Films', 'director': 'Tommy Wiseau', 'review_text': 'If this is a movie, then I am PhD in Artificial Intelligence.', 'user_name': 'alpha_helix'}
{'_id': 'sample_0', 'movie_name': 'Squid Game', 'release_year': 2021, 'producer': 'Siren Pictures Inc', 'director': 'Hwang Dong-hyuk', 'review_text': 'Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!', 'user_name': 'kosmasp'}
{'_id': 'sample_1', 'movie_name': 'The Matrix', 'release_year': 1999, 'producer': 'Lana Wachowski,James McTeiguem,Grant Hill', 'director': 'Lana Wachowski,Lilly Wachowski', 'review_text': 'Best movie of all time !', 'user_name': 'abs_waleedm'}
{'_id': 'sample_4', 'movie_n

In [ ]:
Document(content=batch_data['review_text'], meta=batch_data)

<Document: {'content': ['If this is a movie, then I am PhD in Artificial Intelligence.', 'Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!', 'Best movie of all time !', "They don't make Disney movies like that anymore. I cry every time I watch it.", "C'est une critique de film neutre, mais je n'aime pas non plus Leon.", 'If this is a movie, then I am PhD in Artificial Intelligence.', 'Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!', 'Best movie of all ti

In [ ]:
batch_data = json.load(open('/content/drive/MyDrive/Work/Proteinea Challenge/experimentation/batch_data.json', 'r'))
df = pd.DataFrame.from_dict(batch_data)
detector = Detector()

for index in trange(len(df), desc='Detecting Language'):

  review_text = df.loc[index, 'review_text']

  languages = detector.detect(review_text, aggregated=True)
  lang_primary_type = languages[0][0]

  df.loc[index,'language'] = lang_primary_type

Detecting Language:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
df

,_id,movie_name,release_year,producer,director,review_text,user_name,language
0,sample_3,The Room,2003,Wiseau-Films,Tommy Wiseau,"If this is a movie, then I am PhD in Artificial Intelligence.",alpha_helix,eng
1,sample_0,Squid Game,2021,Siren Pictures Inc,Hwang Dong-hyuk,"Als Fan des koreanischen Kinos war ich sowieso neugierig darauf. Und es hat nicht enttäuscht - überhaupt nicht. Die Schauspielerei ist fantastisch, die Effekte sind großartig ... und die Geschichte ist so ausgereift, wie es nur geht. Mit genug sozialem Kommentar und Kritik, um dies zu einer großartigen Uhr zu machen!",kosmasp,deu
2,sample_1,The Matrix,1999,"Lana Wachowski,James McTeiguem,Grant Hill","Lana Wachowski,Lilly Wachowski",Best movie of all time !,abs_waleedm,eng
3,sample_4,Lion King,1994,"Don Hahn,Alice Dewey,Sarah McArthur","Rob Minkoff,Roger Allers",They don't make Disney movies like that anymore. I cry every time I watch it.,beta_sheet,eng
4,sample_2,Léon: The Professional,1994,Patrice Ledoux,Luc Besson,"C'est une critique de film neutre, mais je n'aime pas non plus Leon.",neu_star,fra


## Language Detection

In [ ]:
detector = Detector()

In [ ]:
languages = detector.detect(dutch_text, aggregated=True)
languages

[('deu', 0.9937790632247925)]

In [ ]:
languages[0][0]

'deu'

## Building Pipeline

- My Algorithm is based on building **Documents** out of the `review_text`, So that I can create a more rapid and reliable technique using **Hay Stack**. 

- These Documents will allow us to switch from using a query by query to a list of documents with metadata sent to multiple models in one pipeline so that we can have our target (a.k.a sentiment analysis for the reviewed movie(s)), rapidly. My Target was to reduce the cost by increasing efficiency by reducing cost, therefore I built a stacked pipeline that flows the `N` number of documents to get translated and classified then stored into **RDS** or **JSON** file.

<center><img src='https://i.imgur.com/q5rKysc.png'/></center>

### (A) Building Custom Node for Language Detection

In [ ]:
class LanguageDetection(BaseComponent):
  outgoing_edges = 3

  def run(self, documents: List[Document]) -> Tuple[Dict[Text, List[Document]], Text]:
    language = documents[0].meta['language']
    if language == "eng":
      return {"documents": documents}, "output_1"
    elif language == "deu" or language == "nld":
      return {"documents": documents}, "output_2"
    elif language == "fra":
      return {"documents": documents}, "output_3"

### (B) Working on Language Translation Models

#### (1) Dutch Transformer Translator

In [ ]:
dutch_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-de-en")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [ ]:
dutch_translator.translate(query=dutch_text)

"As a fan of Korean cinema, I was curious anyway. And it didn't disappoint - not at all. Acting is fantastic, the effects are great... and the story is as mature as it can be. With enough social commentary and criticism to make this a great watch!"

#### (2) French Transformer Translator

In [ ]:
french_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-fr-en")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [ ]:
fre_doc = Document(french_text)

In [ ]:
fre_predicts = french_translator.translate(documents=[fre_doc])
fre_predicts

[<Document: {'content': "It's a neutral film critic, but I don't like Leon either.", 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': 'c86510fa1abd7a7c74e5b18c7fe94245'}>]

In [ ]:
fre_doc.to_dict()['content']

"It's a neutral film critic, but I don't like Leon either."

### (C) Create Custom Node for Document Conversion 

In [ ]:
# class DocumentConverter(BaseComponent):
#   outgoing_edges = 1

#   def run(self, documents: List[Document]):
#     print(documents)
#     return {'documents': documents}, 'output_1'

### (D) Sentiment Analysis

- Since it is a movie sentiment analysis, we know that the emotions of watching movies can affect the review of the clients. Therefore, it will be much more efficient to use a text classifier that is based on emotion or trained on emotional text.

In [ ]:
sent_analysis = TransformersDocumentClassifier(model_name_or_path="valhalla/distilbart-mnli-12-3",
                                               task ="zero-shot-classification",
                                               labels=['good', 'neutral', 'bad'])

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [ ]:
doc = Document(en_text)

In [ ]:
doc

<Document: {'content': 'Best movie of all time !', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '19f2d3173890f47863aed38d03ce8534'}>

In [ ]:
sent_analysis.predict([doc])

[<Document: {'content': 'Best movie of all time !', 'content_type': 'text', 'score': None, 'meta': {'classification': {'sequence': 'Best movie of all time !', 'labels': ['good', 'neutral', 'bad'], 'scores': [0.8598898649215698, 0.1007087379693985, 0.03940136730670929], 'label': 'good'}}, 'embedding': None, 'id': '19f2d3173890f47863aed38d03ce8534'}>]

### Build

In [ ]:
os.environ['HAYSTACK_TELEMETRY_ENABLED'] = "False"

pipe = Pipeline()
pipe.add_node(component=LanguageDetection(),
              name='LanguageDetection',
              inputs=['Query'])
pipe.add_node(component=dutch_translator,
              name='DutchTransformerTranslator',
              inputs=['LanguageDetection.output_2'])
pipe.add_node(component=french_translator,
              name='FrenchTransformerTranslator',
              inputs=['LanguageDetection.output_3'])
# pipe.add_node(component=DocumentConverter(),
#               name="DocumentConverter",
#               inputs=['LanguageDetection.output_1',
#                       'DutchTransformerTranslator',
#                       'FrenchTransformerTranslator'])
pipe.add_node(component=sent_analysis,
              name="SentimentAnalysis",
              inputs=['LanguageDetection.output_1',
                      'DutchTransformerTranslator',
                      'FrenchTransformerTranslator'])
pipe.draw()

## Major Code

### Pipeline

In [ ]:
%%writefile movie_reviewer/pipeline.py
from utils import *

class LanguageDetection(BaseComponent):
  outgoing_edges = 3

  def run(self, documents: List[Document]) -> Tuple[Dict[Text, List[Document]], Text]:
    language = documents[0].meta['language']
    if language == "eng":
      return {"documents": documents}, "output_1"
    elif language == "deu" or language == "nld":
      return {"documents": documents}, "output_2"
    elif language == "fra":
      return {"documents": documents}, "output_3"


class MovieSentimentReviewsPipeline(Pipeline):

  @staticmethod
  def dutch_translator():
    return TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-de-en")
  
  @staticmethod
  def french_translator():
    return TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-fr-en")
  
  @staticmethod
  def sentiment_analysis():
    return TransformersDocumentClassifier(model_name_or_path="valhalla/distilbart-mnli-12-3",
                                          task ="zero-shot-classification",
                                          labels=['good', 'neutral', 'bad']) 
  def __init__(self) -> None:
    super(MovieSentimentReviewsPipeline, self).__init__()

    self.add_node(component=LanguageDetection(),
                  name='LanguageDetection',
                  inputs=['Query'])
    self.add_node(component=MovieSentimentReviewsPipeline.dutch_translator(),
                  name='DutchTransformerTranslator',
                  inputs=['LanguageDetection.output_2'])
    self.add_node(component=MovieSentimentReviewsPipeline.french_translator(),
                  name='FrenchTransformerTranslator',
                  inputs=['LanguageDetection.output_3'])
    self.add_node(component=MovieSentimentReviewsPipeline.sentiment_analysis(),
                  name="SentimentAnalysis",
                  inputs=['LanguageDetection.output_1',
                          'DutchTransformerTranslator',
                          'FrenchTransformerTranslator'])

Overwriting movie_reviewer/pipeline.py


In [ ]:
pipe = MovieSentimentReviewsPipeline()

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [ ]:
pipe.draw()

### Setup Config Function

In [ ]:
%%writefile movie_reviewer/main.py
from utils import *
from pipeline import *

# Initialize pipeline
pipe = MovieSentimentReviewsPipeline()

def config_func(filename: Text,
                output_path: Text=None) -> None:

  def to_dict(documents: Dict[Text, List[Document]]) -> Dict[Text, Text]:
    data = OrderedDict()
    for doc in documents['documents']:
      for key, value in doc.to_dict().items():
        data.setdefault(key, []).append(value)
    # Change the key name to translated text
    data['translated_text'] = data.pop('content')
    data.move_to_end('translated_text', last=False)
    return dict(data)

  batch_data = json.load(open(filename, 'r'))
  df = pd.DataFrame.from_dict(batch_data)
  detector = Detector()

  en_list_of_docs = []
  dutch_list_of_docs = []
  fra_list_of_docs = []

  list_of_docs = []

  def to_rds(data: pd.DataFrame):
    print(" Store Values in RDS Database ".center(100, "="))
    Database_endpoint = os.environ['ENDPOINT']
    Username = os.environ['USER']
    Password = os.environ['PASS']
    database = os.environ['DATABASE']
    try:
      # Create an Engine
      db_data = f'mysql+mysqldb://{Username}:{Password}@{Database_endpoint}:3306/{database}?charset=utf8mb4'
      engine = create_engine(db_data)
      # try:
      #   print("Connecting to "+Database_endpoint)
      #   db = pymysql.connect(host=Database_endpoint,
      #                        port=int(3306),
      #                        user=Username,
      #                        passwd=Password,
      #                        db=database,
      #                        charset="utf8mb4",
      #                        connect_timeout=int(60))
      #   cursor = db.cursor()
      #   print ("Connection successful to "+Database_endpoint)
      # except Exception as e:
      #   print ("Connection unsuccessful due to "+str(e))
      data.to_sql('movie_reviews_predictions', 
                  con=engine,
                  if_exists='append',
                  index=False)
      engine.dispose()
    except Exception as e:
      print(f"Can't connect with RDS, Error: {e}")
      pass

  for index in trange(len(df), desc='Detecting Language'):

    review_text = df.loc[index, 'review_text']
    languages = detector.detect(review_text, aggregated=True)
    lang_primary_type = languages[0][0]

    df.loc[index,'language'] = lang_primary_type

  for index in trange(len(df), desc='Building the Documents'):
    if df.loc[index, 'language'] == 'eng':
      en_list_of_docs.append(Document(content=df.loc[index, 'review_text'],
                                      meta=df.loc[index, :].to_dict()))
    elif df.loc[index, 'language'] == "deu" or df.loc[index, 'language'] == "nld":
      dutch_list_of_docs.append(Document(content=df.loc[index, 'review_text'],
                                      meta=df.loc[index, :].to_dict()))
    elif df.loc[index, 'language'] == 'fra':
      fra_list_of_docs.append(Document(content=df.loc[index, 'review_text'],
                                      meta=df.loc[index, :].to_dict()))
  
  for document in tqdm([en_list_of_docs, dutch_list_of_docs, fra_list_of_docs], desc='Deploy pred'):
    list_of_docs.append(to_dict(pipe.run(documents=document)))
  
  # Printing the values
  # pprint(list_of_docs, indent=2)
  
  if output_path:
    # Save the file
    with open(os.path.join(output_path, 'prediction_metadata.json'), 'w') as json_file:
      json.dump(list_of_docs, json_file, indent=2)
  else:
    # Write to RDS
    to_rds(pd.DataFrame.from_dict(list_of_docs))

  print(' Successfully saved data '.center(100, '='))
  

if __name__ == "__main__":
  fire.Fire(config_func)

Overwriting movie_reviewer/main.py


In [ ]:
config_func('/content/drive/MyDrive/Work/Proteinea Challenge/experimentation/batch_data.json',
            # '/content/drive/MyDrive/Work/Proteinea Challenge/experimentation'
            )

Detecting Language:   0%|          | 0/5 [00:00<?, ?it/s]

Building the Documents:   0%|          | 0/5 [00:00<?, ?it/s]

Deploy pred:   0%|          | 0/3 [00:00<?, ?it/s]

                                                                                                                                                                                                                                            translated_text        content_type               score                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 meta           embedding                                                                                                      id
0                                                                                  

In [ ]:
!python movie_reviewer/main.py \
 --filename "/content/drive/MyDrive/Work/Proteinea Challenge/experimentation/batch_data.json" \
 --output_path .

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.telemetry -  Telemetry has been disabled.
INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
Detecting Language: 100% 5/5 [00:00<00:00, 17.18it/s]
Building the Docum

### Dockerfile

In [ ]:
%%writefile movie_reviewer/Dockerfile
FROM nvidia/cuda:11.0-cudnn8-runtime-ubuntu18.04

ENV LC_ALL=C.UTF-8
ENV LANG=C.UTF-8

WORKDIR /movie_reviewer
COPY . .

# Install software dependencies
RUN apt-get update && apt-get install -y \
    software-properties-common

RUN add-apt-repository ppa:deadsnakes/ppa

RUN apt-get update && apt-get install -y \
    cmake \
    unzip \
    curl \
    git \
    libpoppler-cpp-dev \
    libtesseract-dev \
    pkg-config \
    poppler-utils \
    python3-pip \
    python3.7 \
    python3.7-dev \
    python3.7-distutils \
    default-libmysqlclient-dev \
 	build-essential \
    swig \
    tesseract-ocr \
    wget && \
rm -rf /var/lib/apt/lists/*


# Set default Python version
RUN update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1 && \
    update-alternatives --set python3 /usr/bin/python3.7


# Install PyTorch for CUDA 11
RUN pip3 install --no-cache-dir torch==1.10.2+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# Install package
RUN apt-get install -y poppler-utils
RUN pip3 install --upgrade pip
RUN pip3 install --no-cache-dir -r requirements.txt
RUN python3 -m pip install cffi

# Run Required models to be cached --> Translator & Sentiment Analysis models
RUN python3 -c "from haystack.nodes import TransformersTranslator, TransformersDocumentClassifier;TransformersTranslator(model_name_or_path='Helsinki-NLP/opus-mt-de-en');TransformersTranslator(model_name_or_path='Helsinki-NLP/opus-mt-fr-en');TransformersDocumentClassifier(model_name_or_path='valhalla/distilbart-mnli-12-3', task ='zero-shot-classification', labels=['good', 'neutral', 'bad'])"


EXPOSE 80
ENV HAYSTACK_DOCKER_CONTAINER="HAYSTACK_GPU_CONTAINER"

ENTRYPOINT ["python3", "./main.py"]

Overwriting movie_reviewer/Dockerfile
